In [14]:
%load_ext autoreload
%reload_ext autoreload
#autoreload 2：装入所有 %aimport 不包含的模块。
%autoreload 2
#%matplotpy
import numpy as np
from scuteegfe.mne_features_wrapper.feature_wrapper import Feature
rng = np.random.RandomState(42)
n_epochs, n_channels, n_times = 2, 2,1800 
X = rng.randn(n_epochs, n_channels, n_times)
feat1=Feature(X,sfreq=250,selected_funcs={'decorr_time'})
print(feat1.features.shape)
# feat2=Feature(X,sfreq=250,selected_funcs={'pow_freq_bands'})juju
# print(feat2.features.shape)
select_para=dict({'EMD__EMD_times': 1,
                  'EMD__EMD_params': 6})
feat=Feature(X,sfreq=250,selected_funcs={'EMD'},funcs_params=select_para)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(2, 2, 1)
(2, 2, 10500)


In [2]:
# test EMD

In [ ]:
#test hos
import os,sys
import numpy as np
from scuteegfe.HOSA.conventional.bicoherence import bicoherence
import scipy.io as scio
load_mat = scio.loadmat('qpc.mat')['zmat']
y=load_mat[0,:][np.newaxis,:]
bicoherence(y, nfft=None, wind=None, nsamp=None, overlap=None)